In [232]:
import numpy as np
import math
from scipy.stats import poisson, norm, bernoulli, gamma
from scipy.special import psi

#nneuron = 41
#sprefs = np.linspace(-60, 60, nneuron)

nneuron = 21
sprefs = np.linspace(-20, 20, nneuron)

sig0_sq      = float(3**2)
sig1_sq      = float(12**2)
sigtc_sq     = float(10**2)
ndatapergain = 3000
mu = 0
gain = 15

def generate_popcode_data(ndatapergain, nneuron, sig0_sq, sig1_sq, sigtc_sq, gain, noise):
    s = np.random.normal(mu, np.sqrt(sig0_sq), ndatapergain)
    sprefs_data = np.tile(sprefs, (ndatapergain, 1))
    r = gain * np.exp(-np.square((np.transpose(np.tile(s, (nneuron, 1))) - sprefs_data))/(2 * sigtc_sq))
    if noise == "poisson":
        r = np.random.poisson(r) + 0.0
    return r, s

In [ ]:
#simple version
r, actual = generate_popcode_data(ndatapergain, nneuron, sig0_sq, sig1_sq, sigtc_sq, 1, "poisson")
def norm_g(zTw):
    return zTw + 3

def relu(zTw):
    return max(0.000000000000001, zTw)

def g_norm(z, loc, scale):
    d_loc = (z - loc)/scale
    d_scale = -(1/(2 * scale)) + ((z - loc)**2)/(2 * scale**2)
    return [d_loc, d_scale]

L = 1
s = 100
z = [None]*L
W = [None]*L
p_z = [None]*(L + 1)
q_z = [None]*L
p_W = [None]*L
q_W = [None]*L
g = [None]*L
g_W = [None]*L

xi_normal = np.random.rand(nneuron, 2) #[mu, sigma]
#train_data = r[1:3000]
train_data = r[1:3000]
actual = actual[1:3000]
rho = .001
rhoW = .001
MeanSquare = [None] * L
for l in range(L):
    MeanSquare[l] = [None] * 2
    MeanSquare[l][0] = [None] * len(xi_normal)
    print MeanSquare[l][0]
    for m in range(len(MeanSquare[l][0])):
        MeanSquare[l][0][m] = [0] * 2  

num = 1
for i in range(len(train_data)):
    n = train_data[i]
    true = actual[i]
    converged = False
    lam_normal = [0, 1] #[mu, sigma]
    MeanSquare[l][1] = [0] * 2
    while not converged:
        #normal
        z_loc, z_scale = lam_normal
        z[0] = np.random.normal(loc=z_loc, scale=z_scale, size=s)
        W[0] = [None]*len(xi_normal)
        for w in range(len(xi_normal)):
            W_loc, W_scale = xi_normal[w]
            W[0][w] = np.random.normal(loc=W_loc, scale=W_scale, size=s).tolist()
        p_z[0] = [None] * s
        p_z[1] = [None] * s
        q_z[0] = [None] * s
        g[0] = [None] * s
        g_W[0] = [None] * s
        params = z[0] * W[0]
        params_a = np.array(params)
        for t in range(s):
            lam_p = params_a[:, t]
            p_z[0][t] = [None] * len(lam_p)
            g_W[0][t] = [None] * len(lam_p)
            for i in range(len(n)):
                p_z[0][t][i] = poisson.pmf(n[i], relu(lam_p[i]))
                g_W[0][t][i] = g_norm(W[0][i][t], W_loc, W_scale)
            np.concatenate(g_W[0][t])
            q_z[0][t] = norm.pdf(x=z[0][t], loc=z_loc, scale=z_scale)
            g[0][t] = g_norm(z[0][t], z_loc, z_scale)
            p_z[1][t] = [norm.pdf(x=z[0][t], loc=mu, scale=np.sqrt(sig0_sq))]
        p_z[0] = np.log(p_z[0])
        p_z[1] = np.log(p_z[1])
        q_z[0] = np.log(q_z[0])
        
        l = 0
        S = [None] * L
        T = [None] * L
        for l in range(L):
            S[l] = [None] * s
            T[l] = [None] * s
            for t in range(s):
                pzl = np.sum(p_z[l][t])
                glk = np.asarray(g[l][t])
                S[l][t] = glk * (pzl + p_z[l+1][t] + q_z[l][t]) #???
                S[l][t] = S[l][t].tolist()
                W_a = np.array(W)
                lenW = len(W_a[l, :, t])
                T[l][t] = [None] * lenW
                for j in range(lenW):
                    gWlj = np.array(g_W[l][t][j])
                    #T[l][j] = gWlj * pzl #???
                    T[l][t][j] = gWlj * p_z[l][t][j] #???
                    T[l][t][j] = T[l][t][j].tolist()
        S = np.array(S)
        T = np.array(T)
        g_Smu = np.mean(S[0, :, 0])
        g_Ssigma = np.mean(S[0, :, 1])
        MeanSquare[0][1][0] = (.9 * MeanSquare[0][1][0]) + (.1 * (g_Smu ** 2))
        MeanSquare[0][1][1] = (.9 * MeanSquare[0][1][1]) + (.1 * (g_Ssigma ** 2))
        lam_normal[0] += rho * (1/np.sqrt(MeanSquare[0][1][0])) * g_Smu
        lam_normal[1] += rho * (1/np.sqrt(MeanSquare[0][1][1])) * g_Ssigma
        lam_normal[1] = relu(lam_normal[1])
        g_Tmu = [None] * len(xi_normal)
        g_Tsigma = [None] * len(xi_normal)
        for m in range(len(xi_normal)):
            g_Tmu[m] = np.mean(T[0, :, m, 0])
            g_Tsigma[m] = np.mean(T[0, :, m, 1])
            MeanSquare[0][0][m][0] = (.9 * MeanSquare[0][0][m][0]) + (.1 * (g_Tmu[m] ** 2))
            MeanSquare[0][0][m][1] = (.9 * MeanSquare[0][0][m][1]) + (.1 * (g_Tsigma[m] ** 2))
            xi_normal[m][0] += rhoW * (1/np.sqrt(MeanSquare[0][0][m][0])) * g_Tmu[m]
            xi_normal[m][1] += rhoW * (1/np.sqrt(MeanSquare[0][0][m][1])) * g_Tsigma[m]
            xi_normal[m][1] = relu(xi_normal[m][1])
        num += 1
        if num > 100:
            converged = True
            print "x", n
            print "lambda", lam_normal
            print "true", true
            print "xi", xi_normal
            num = 1
            rhoW = rhoW * .99
            rho = .001

In [237]:
print len(xi_normal[0])

2
